In [7]:
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()
GROQ_API_KEY=os.getenv("GROQ_API_KEY")


In [9]:
os.environ["GROQ_API_KEY"]=GROQ_API_KEY

In [10]:
#Loading the PDF data
from langchain.document_loaders import PyPDFLoader

In [11]:
%pwd

'/Users/anujrai/Desktop/AI/Langchain-Proj/research'

In [12]:
%cd ..

/Users/anujrai/Desktop/AI/Langchain-Proj


In [13]:
%pwd

'/Users/anujrai/Desktop/AI/Langchain-Proj'

In [14]:
file_path="data/SDG.pdf"
loader = PyPDFLoader(file_path)
data=loader.load()

In [15]:
data

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2020-09-15T15:08:34-04:00', 'author': 'Susana.Castillo', 'moddate': '2020-09-15T15:08:34-04:00', 'title': 'sdgs_targets_overview_resource.pdf', 'source': 'data/SDG.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, page_content='4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men

In [22]:
len(data)
type(data)

list

In [18]:
#Copying the pages into one place 

question_gen=""
for page in data:
    question_gen+=page.page_content


In [ ]:
question_gen


'4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  Target 1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable  Target 1.4 By 2030, ensure that all me

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
splitter_ques_gen= RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=200
)

In [30]:
chunk_ques_gen= splitter_ques_gen.split_text(question_gen)

In [31]:
type(chunk_ques_gen)

list

In [32]:
len(chunk_ques_gen)

4

In [33]:
#Converting the chunks into documents
from langchain.docstore.document import Document

In [34]:
document_ques_gen=[Document(page_content=t)for t in chunk_ques_gen]

In [35]:
splitter_ans_gen= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [37]:
chunk_ans_gen= splitter_ans_gen.split_documents(document_ques_gen)

In [38]:
chunk_ans_gen

[Document(metadata={}, page_content='4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  Target 1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable  Ta

In [39]:
len(chunk_ans_gen)

51

In [40]:
from langchain_groq import ChatGroq

In [42]:
llm_ques_gen_pipeline = ChatGroq(model="llama-3.3-70b-versatile")

In [43]:
prompt_template="""
You are an expert at creating questions based on materials and documentation provided.
Your goal is to prepare user for their exams and tests.

------
{text}
------

Create questions that will prepare user for their exams and tests.
Make sure not to loose any important information.

QUESTIONS:
"""

In [44]:
from langchain.prompts import PromptTemplate

In [45]:
PROMPT_QUESTIONS= PromptTemplate(template=prompt_template,input_variables=['text'])

In [ ]:
refine_template = ("""
You are an expert at creating questions based on materials and documentation provided.
Your goal is to prepare user for their exams and tests.
We have received some practice questions to a certain extent:{existing_answer}.
We have the option to refine the questions or add new one (only if neccessary) with some more context below.

------------
{text}
------------

Given the new content, refine the orginial questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
""")

In [ ]:
REFINE_PROMPT_TEMPLATE= PromptTemplate(
    input_variables=["existing_answers","text"],
    template=refine_template,
)

In [48]:
from langchain.chains.summarize import load_summarize_chain

In [49]:
ques_gen_chain=load_summarize_chain(
    llm=llm_ques_gen_pipeline,
    chain_type="refine",
    verbose=True,
    question_prompt=PROMPT_QUESTIONS,
    refine_prompt=REFINE_PROMPT_TEMPLATE,

)

In [50]:
ques=ques_gen_chain.run(document_ques_gen)

print(ques)

/var/folders/00/6gmykrw13ylb0nr60ggprq8c0000gn/T/ipykernel_91450/1135716077.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques=ques_gen_chain.run(document_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on materials and documentation provided.
Your goal is to prepare user for their exams and tests.

------
4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimension

In [51]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [52]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/00/6gmykrw13ylb0nr60ggprq8c0000gn/T/ipykernel_91450/3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/opt/anaconda3/envs/LangProj/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
from langchain.vectorstores import FAISS

In [56]:
vector_store= FAISS.from_documents(chunk_ans_gen,embeddings)

In [57]:
llm_answer_gen=ChatGroq(model="llama-3.3-70b-versatile")

In [58]:
ques

"Based on the provided content, I have refined the original questions to better reflect the new context. Here are the refined questions:\n\n**Goal 14: Conserve and Sustainably Use the Oceans, Seas, and Marine Resources**\n\n1. By 2025, what is the goal of preventing and significantly reducing marine pollution, and how will it be achieved?\n2. How will marine and coastal ecosystems be protected and restored, as stated in Target 14.2?\n3. What is the goal of Target 14.3, and how will the impacts of ocean acidification be minimized?\n4. By 2020, what is the goal of regulating harvesting and ending overfishing, as stated in Target 14.4?\n5. What is the goal of Target 14.7, and how will the economic benefits to small island developing States and least developed countries from the sustainable use of marine resources be increased?\n\n**Goal 15: Protect, Restore, and Promote Sustainable Use of Terrestrial Ecosystems**\n\n1. By 2020, what is the goal of ensuring the conservation, restoration, a

In [59]:
ques_list=ques.split('\n')

In [60]:
ques_list

['Based on the provided content, I have refined the original questions to better reflect the new context. Here are the refined questions:',
 '',
 '**Goal 14: Conserve and Sustainably Use the Oceans, Seas, and Marine Resources**',
 '',
 '1. By 2025, what is the goal of preventing and significantly reducing marine pollution, and how will it be achieved?',
 '2. How will marine and coastal ecosystems be protected and restored, as stated in Target 14.2?',
 '3. What is the goal of Target 14.3, and how will the impacts of ocean acidification be minimized?',
 '4. By 2020, what is the goal of regulating harvesting and ending overfishing, as stated in Target 14.4?',
 '5. What is the goal of Target 14.7, and how will the economic benefits to small island developing States and least developed countries from the sustainable use of marine resources be increased?',
 '',
 '**Goal 15: Protect, Restore, and Promote Sustainable Use of Terrestrial Ecosystems**',
 '',
 '1. By 2020, what is the goal of ensu

In [62]:
from langchain.chains import RetrievalQA

In [68]:
answer_generation_chain= RetrievalQA.from_chain_type(
    llm=llm_answer_gen,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_type="similarity")
)

In [69]:
for question in ques_list:
    print("Question:",question)
    answer=answer_generation_chain.run(question)
    print("Answer:",answer)
    print("-----------------------------------------------")

    with open("answers.txt","a") as f:
        f.write("Question:" + question + "\\n")
        f.write("Answer:"+answer+"\\n")
        f.write("-----------------------------------------")

Question: Based on the provided content, I have refined the original questions to better reflect the new context. Here are the refined questions:
Answer: You haven't provided any questions yet. Please go ahead and ask your refined questions based on the provided content, and I'll do my best to help.
-----------------------------------------------
Question: 
Answer: It looks like you didn't ask a question. Please go ahead and ask your question, and I'll do my best to answer it based on the provided context. If I don't know the answer, I'll let you know that too.
-----------------------------------------------
Question: **Goal 14: Conserve and Sustainably Use the Oceans, Seas, and Marine Resources**
Answer: The targets for Goal 14 are:

1. **Target 14.1**: By 2025, prevent and significantly reduce marine pollution of all kinds, in particular from land-based activities, including marine debris and nutrient pollution.
2. **Target 14.2**: By 2020, sustainably manage and protect marine and c